In [1]:
import time
import threading

from nvitop import ResourceMetricCollector
from nvitop import Device, GpuProcess, NA, colored

def collect_in_background(
    on_collect,
    collector=None,
    interval=None,
    *,
    on_start=None,
    on_stop=None,
    tag='',
    start=True,
):
    if collector is None:
        collector = ResourceMetricCollector()
    if interval is None:
        interval = collector.interval
    interval = min(interval, collector.interval)

    def target():
        if on_start is not None:
            on_start(collector)
        try:
            with collector(tag):
                try:
                    while on_collect(collector.collect()):
                        time.sleep(interval)
                except KeyboardInterrupt:
                    pass
        finally:
            if on_stop is not None:
                on_stop(collector)

    daemon = threading.Thread(target=target, daemon=True)
    if start:
        daemon.start()
    return daemon

In [2]:
import logging

def set_logger():
    # create logger
    logger = logging.getLogger(__name__)
    
    handlers = logger.handlers
    [logger.removeHandler(x) for x in handlers]
    
    # set log level for all handlers to debug
    logger.setLevel(logging.DEBUG)

    # create console handler and set level to debug
    # best for development or debugging
    consoleHandler = logging.StreamHandler()
    consoleHandler.setLevel(logging.DEBUG)

    # create formatter
    formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

    # add formatter to ch
    consoleHandler.setFormatter(formatter)

    # add ch to logger
    logger.addHandler(consoleHandler)

    ###################
    # USE LOGGER
    ###################

    # example usage

    return logger

In [3]:
logger = set_logger()

logger.debug('debug message')
logger.info('info message')
logger.warning('warn message')
logger.error('error message')
logger.critical('critical message')

2022-11-13 04:48:42,669 - __main__ - DEBUG - debug message
2022-11-13 04:48:42,670 - __main__ - INFO - info message
2022-11-13 04:48:42,671 - __main__ - WARNING - warn message
2022-11-13 04:48:42,672 - __main__ - ERROR - error message
2022-11-13 04:48:42,673 - __main__ - CRITICAL - critical message


In [4]:
# base_metrics = collector.collect()
# metrics_vals = [x.split(' ')[0] for x in base_metrics.keys() if 'mean' in x and not 'load_average' in x]

from tqdm.notebook import tqdm

def set_tqdm():
    tempo_collector = ResourceMetricCollector()
    with tempo_collector(tag=''):
        base_metrics = tempo_collector.collect()
        
    metrics_vals = [x.split(' ')[0] for x in base_metrics.keys() if 'mean' in x and not 'load_average' in x]    

    tempo_collector.stop()

    tqdms = {}
    for x in metrics_vals:
        tqdms[x] = tqdm(total=0, position=2, bar_format='{desc}')
        tqdms[x].set_description_str(f"{x} - {0}")
    
    return tqdms

In [5]:
li_logger = []

tqdms = set_tqdm()

def on_collect(metrics):
    for key in metrics.keys():
        # print(key)
        if 'mean' in key and not 'load_average' in key:
            tqdms[key.split(' ')[0]].set_description_str(f"{key.split(' ')[0]} - {metrics[key]}")
                
    # if logger.is_closed():  # closed manually by user
    #     return False
    # logger.log(metrics)
    # logger.info(metrics)
    li_logger.append(metrics)
    return True

def on_stop(collector):
    print('logging is completed')
    # if not logger.is_closed():
    #     logger.close()  # cleanup

background_collector = ResourceMetricCollector()
daemon = collect_in_background(on_collect, background_collector, interval=5.0, on_stop=on_stop)

for epoch in range(100):
    time.sleep(1)
    # Do something
    for batch in range(100):
        # Do something
        pass

for x in tqdms:
    tqdms[x].close()